<a href="https://colab.research.google.com/github/jww2145/heart-disease/blob/main/Heart_Disease_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Proposal**

We plan on using [this](https://https://www.kaggle.com/datasets/cherngs/heart-disease-cleveland-uci) dataset found on Kaggle regarding the presence of heart disease of 297 patients in a Cleaveland Clinic. In the introductory paper, led by Professor Robert Detrano, it was mentioned the Kaggle dataset was used to compare actual presence of heart disease in patients with results gained from applying a Bayesian algorithm. As this is our first time utilizing machine learning, our first step would be to follow the methodology laid out by Professor Detrano and compare our model to his model, presented in the research. This way, we can ensure that our dataset is trained correctly before moving on.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
#import kaggle dataset

from google.colab import drive

#uncomment when need to remount

#drive.mount('/content/drive')

#change the file path when trying to run from your local machine
df = pd.read_csv('/content/drive/My Drive/Heart Disease/heart_cleveland_upload.csv')
print(df.shape)

(297, 14)


In [ ]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


Because the dataset found on Kaggle has been pre-cleaned and processed, we do not need to worry about verifying the data on our own.

In [ ]:
# condition is our target column - as it indicates the presence of heart disease in a patient

In [ ]:
conditions = df.pop("condition")

In [ ]:
from sklearn.model_selection import train_test_split

# split data into random train and test subsets

X_train, X_test, Y_train, Y_test = train_test_split(df, conditions)

In [ ]:
# Creating the random forests with 18 trees, which is the sqrt of 297 rounded up.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=500)
clf = clf.fit(X_train, Y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, y_pred)

roc_auc = auc(false_positive_rate, true_positive_rate)

roc_auc

0.811692969870875

The AUROC at 18 trees is greater than .8, which means our model has some degree class separation capacity.